In [ ]:

!pip install jiwer 
!pip install librosa
!pip install torchaudio
!pip install datasets==1.14
!pip install transformers==4.11.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset, load_dataset_builder,get_dataset_config_names,get_dataset_split_names, load_metric, Audio
dt_AutoConfig_ = get_dataset_config_names('openslr')

# dt_AutoConfig_ = get_dataset_config_names('superb','asr')

In [ ]:
dt_AutoConfig_

['SLR32',
 'SLR35',
 'SLR36',
 'SLR41',
 'SLR42',
 'SLR43',
 'SLR44',
 'SLR52',
 'SLR53',
 'SLR54',
 'SLR63',
 'SLR64',
 'SLR65',
 'SLR66',
 'SLR69',
 'SLR70',
 'SLR71',
 'SLR72',
 'SLR73',
 'SLR74',
 'SLR75',
 'SLR76',
 'SLR77',
 'SLR78',
 'SLR79',
 'SLR80',
 'SLR86']

In [ ]:
dt_builder = load_dataset_builder('openslr','SLR70')
dt_builder.info.description

'OpenSLR is a site devoted to hosting speech and language resources, such as training corpora for speech recognition,\nand software related to speech recognition. We intend to be a convenient place for anyone to put resources that\nthey have created, so that they can be downloaded publicly.\n'

In [ ]:
get_dataset_split_names('openslr','SLR70')

['train']

In [ ]:
dataset_ = load_dataset('openslr','SLR70',split='train')  # ,streaming=True)

In [ ]:
dataset_ = dataset_.train_test_split(test_size=0.2, shuffle=True)

In [ ]:
import os
 
# Get the list of all files and directories
path = "/root/.cache/huggingface/datasets/open_slr"   #
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

Files and directories in ' /root/.cache/huggingface/datasets/open_slr ' :
['SLR70']


In [ ]:
from google.colab import files
files.download("/root/.cache/huggingface/datasets/open_slr")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
dataset_

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 2687
    })
    test: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 672
    })
})

In [ ]:
dataset_ = dataset_.cast_column("audio", Audio(sampling_rate=16000))  # the pretrained model sampling rate is 16Khz so,castaudiocolumn to 16Khz

In [ ]:
dataset_['train'][0] #no unwanted columns to remove

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/d58d4905301c2fedf94e7399eb0d285d2ceaa0dee42101795793fda018cf88e0/ngm_09697_01399735206.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/d58d4905301c2fedf94e7399eb0d285d2ceaa0dee42101795793fda018cf88e0/ngm_09697_01399735206.wav',
  'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
  'sampling_rate': 16000},
 'sentence': 'Some discourses of a certain vile persuasion drug and trick the soul'}

In [ ]:
#remove special characters from the sentence
import re
characters_2_remove = '[\,\?\.\!\-\;\:\"\_\é\(\)\’\–\—\"\ẹ]'
chrs_= '[\xa0\ẹ\\\]'

def remove_special_characters(data):
  data['sentence']= re.sub(characters_2_remove,'',data['sentence']).lower()+' '
  data['sentence']= re.sub(chrs_,'',data['sentence']).lower()+' '
  return data

In [ ]:
dataset_ = dataset_.map(remove_special_characters)

  0%|          | 0/2687 [00:00<?, ?ex/s]

  0%|          | 0/672 [00:00<?, ?ex/s]

In [ ]:

def complete_characters_frdata(data):
  complete_sentence = ' '.join(data['sentence'])
  complete_vocab_list = list(set(complete_sentence))
  return {'vocab':[complete_vocab_list], 'sentence' :[complete_sentence]}

In [ ]:
vc = dataset_.map(complete_characters_frdata,
                  batched=True,
                  batch_size=-1,
                  keep_in_memory=True,
                  remove_columns=dataset_.column_names['train'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
converted_vocab_list = list(set(vc['train']['vocab'][0]) | set(vc['test']['vocab'][0]))
converted_vocab_dict =  {voc:indx for indx,voc in enumerate(converted_vocab_list)}

In [ ]:
converted_vocab_dict

{'u': 0,
 '9': 1,
 'a': 2,
 'x': 3,
 'i': 4,
 'p': 5,
 'y': 6,
 't': 7,
 'z': 8,
 '1': 9,
 '3': 10,
 '0': 11,
 'l': 12,
 'v': 13,
 'd': 14,
 'm': 15,
 'c': 16,
 '5': 17,
 'h': 18,
 'f': 19,
 'b': 20,
 'q': 21,
 'j': 22,
 'k': 23,
 '6': 24,
 'r': 25,
 'n': 26,
 ' ': 27,
 'w': 28,
 '2': 29,
 "'": 30,
 '8': 31,
 'e': 32,
 '4': 33,
 'g': 34,
 's': 35,
 'o': 36,
 '7': 37}

In [ ]:
converted_vocab_dict['|'] = converted_vocab_dict[' ']
del converted_vocab_dict[' ']
# unk and pad token for CTC algorithm
converted_vocab_dict['[unq]'] = len(converted_vocab_dict)
converted_vocab_dict['[padded]'] = len(converted_vocab_dict)
print(converted_vocab_dict)

{'u': 0, '9': 1, 'a': 2, 'x': 3, 'i': 4, 'p': 5, 'y': 6, 't': 7, 'z': 8, '1': 9, '3': 10, '0': 11, 'l': 12, 'v': 13, 'd': 14, 'm': 15, 'c': 16, '5': 17, 'h': 18, 'f': 19, 'b': 20, 'q': 21, 'j': 22, 'k': 23, '6': 24, 'r': 25, 'n': 26, 'w': 28, '2': 29, "'": 30, '8': 31, 'e': 32, '4': 33, 'g': 34, 's': 35, 'o': 36, '7': 37, '|': 27, '[unq]': 38, '[padded]': 39}


In [ ]:
converted_vocab_dict

{'u': 0,
 '9': 1,
 'a': 2,
 'x': 3,
 'i': 4,
 'p': 5,
 'y': 6,
 't': 7,
 'z': 8,
 '1': 9,
 '3': 10,
 '0': 11,
 'l': 12,
 'v': 13,
 'd': 14,
 'm': 15,
 'c': 16,
 '5': 17,
 'h': 18,
 'f': 19,
 'b': 20,
 'q': 21,
 'j': 22,
 'k': 23,
 '6': 24,
 'r': 25,
 'n': 26,
 'w': 28,
 '2': 29,
 "'": 30,
 '8': 31,
 'e': 32,
 '4': 33,
 'g': 34,
 's': 35,
 'o': 36,
 '7': 37,
 '|': 27,
 '[unq]': 38,
 '[padded]': 39}

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor,Wav2Vec2Processor,AutoConfig

# model_checkPoint= 'facebook/wav2vec2-large-960h-lv60-self'
model_checkPoint= 'facebook/wav2vec2-base-960h'
batch_size = 32
AutoFeatureExtractor_ = AutoFeatureExtractor.from_pretrained(model_checkPoint)

AutoConfig_ = AutoConfig.from_pretrained(model_checkPoint) # load pretrained model and to modify model attributes 

Tokenizer_type = AutoConfig_.model_type if AutoConfig_.tokenizer_class is None else None 

AutoConfig_ = AutoConfig_ if AutoConfig_.tokenizer_class is not None else None   # other model attributes are randomly initialized

#tokenizer 
# vocabulary to output json file
import json
with open('vocab.json','w') as file:
  json.dump(converted_vocab_dict,file)

with open('/content/drive/MyDrive/data/vocab.json','w') as f:
  json.dump(converted_vocab_dict,f)

# vocab.json instantiate to a tokenizer object
# preprocessing text into an array of numbers as inputs 

tokenizer = AutoTokenizer.from_pretrained('./', # vocab.json file
                              config = AutoConfig_,
                               tokenizer_type=Tokenizer_type,
                               unk_token='[unq]',
                               pad_token='[padded]',
                               word_delimiter='|')  # makes input_ids(tokens numerical presentation) and attention_mask (which tokens should be attended to )
#processor
processor=Wav2Vec2Processor(feature_extractor=AutoFeatureExtractor_, #modelcheckpoint featureextractor
                                     tokenizer=tokenizer) #modelcheckpoint+vocab.json+(ctc)

                                     

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
ex = tokenizer("We are very happy")
print(ex)

{'input_ids': [38, 32, 27, 2, 25, 32, 27, 13, 32, 25, 6, 27, 18, 2, 5, 5, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
for i in range(350,360):
  print(dataset_['train'][i]['sentence'])
  print(dataset_['train'][i]['audio']['sampling_rate'])
  import IPython
  import numpy as np
  import time
  #IPython.display.Audio(data=np.asarray(dataset_['train'][i]['audio']['array']), autoplay=True,rate=48000)
  #time.sleep(10)
  print('-'*20)

asian lacquer dries to a very hard and smooth surface layer which is durable waterproof and attractive to feel and look at  
16000
--------------------
it's maths if you prefer british english  
16000
--------------------
florida memorial university has eight academic divisions  
16000
--------------------
you can get answers just for you too  
16000
--------------------
gaiman believed his copyrighted work was being infringed upon which violated their original oral agreement  
16000
--------------------
here's one for the joystick generation  
16000
--------------------
the men dress with a kaftan on jalabiya  
16000
--------------------
if you want to delete the draft just double tap the google assistant touch surface  
16000
--------------------
please make sure that the text you send is a shorter representation of the audio  
16000
--------------------
it is not uncommon to see the women decorate their hair with bead hair accessories as well as cowrie shells  
16000
---------------

In [ ]:
i = 350
print(dataset_['train'][i]['sentence'])
print(dataset_['train'][i]['audio']['sampling_rate'])
import IPython
import numpy as np
import time
IPython.display.Audio(data=np.asarray(dataset_['train'][i]['audio']['array']), autoplay=True,rate=16000)


asian lacquer dries to a very hard and smooth surface layer which is durable waterproof and attractive to feel and look at  
16000


In [ ]:
# prepare data
def data_preparation(data):
  audio = data['audio']
  data['input_values'] = processor(audio['array'],sampling_rate=audio['sampling_rate']).input_values[0]
  data['input_length_'] = len(data['input_values'])

  with processor.as_target_processor(): # use wav2vecprocessor
    data['labels'] = processor(data['sentence']).input_ids  #numerical representations 
  return data

modified_dataset_ = dataset_.map(data_preparation,remove_columns=dataset_.column_names['train'], num_proc=4) # speed up map by increasing number of process i.e. num_proc


In [ ]:
modified_dataset_ 

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length_', 'labels'],
        num_rows: 2687
    })
    test: Dataset({
        features: ['input_values', 'input_length_', 'labels'],
        num_rows: 672
    })
})

In [ ]:
modified_dataset_

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length_', 'labels'],
        num_rows: 2687
    })
    test: Dataset({
        features: ['input_values', 'input_length_', 'labels'],
        num_rows: 672
    })
})

In [ ]:
modified_dataset_['train'][0]['input_values'] 

[0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.000630015623755753,
 0.00063001

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.models.wav2vec2.modeling_wav2vec2 import Wav2Vec2ForCTC
from transformers.data import data_collator
import torch

# from transformers.tokenization_utils_base import PaddingStrategy
#from ..utils import PaddingStrategy
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding: # pad the inputs received
  processor: Wav2Vec2Processor
  padding: Union[bool,str]= True  #,PaddingStrategy]= True #longest
  max_length :Optional[int] =None   #maximum length of the returned list and optionally padding length
  max_length_labels:Optional[int] = None 
  pad_to_multiple_of : Optional[int] = None    # set will pad the sequence to a multiple of the providedvalue
  pad_to_multiple_of_labels : Optional[int] = None 
  return_tensors: str= 'pt'      #type of tensor
  def __call__(self,features: List[Dict[str,Union[List[int],torch.Tensor]]]) -> Dict[str,torch.Tensor]:
   
    # split inputs and labels since they have to be of different lengths and need different padding methods
    input_features= [{'input_values':feature['input_values']} for feature in features]
    label_features = [{'input_ids':feature['labels']} for feature in features]
    
    
    batch = self.processor.pad(
        input_features,
        padding=self.padding,
        max_length=self.max_length,
        pad_to_multiple_of=self.pad_to_multiple_of,
        return_tensors='pt',
        )
    
    with self.processor.as_target_processor(): # wav2vecprocessor
      labels_batch=self.processor.pad(
          label_features,
          padding=self.padding,
          max_length= self.max_length_labels,
          pad_to_multiple_of = self.pad_to_multiple_of_labels,
          return_tensors = 'pt',
      )
    
    # replace padding with -100 to ignore loss corectly

    labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)

    batch['labels']= labels

    return batch

data_collator= DataCollatorCTCWithPadding(processor=processor,padding=True)

In [ ]:
# Evaluate
# Word Error Rate metric is used to evaluate model performance

wer_metric = load_metric('wer')

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [ ]:
# function that passes predictions and labels to compute to calculate the WER
from transformers import TrainingArguments, Trainer,AutoModelForCTC

import numpy as np

def compute_metrics(pred):
  
  pred_logits=pred.predictions
  pred_ids= np.argmax(pred_logits, axis=-1) 

  pred.label_ids[pred.label_ids==-100] = processor.tokenizer.pad_token_id

  pred_str = processor.batch_decode(pred_ids)

  label_str = processor.batch_decode(pred.label_ids, group_tokens= False)

  wer = wer_metric.compute(predictions= pred_str,references= label_str)

  return {'wer':wer}

In [ ]:
# Wav2Vec2ForCTC
model = AutoModelForCTC.from_pretrained(model_checkPoint,
                                       ctc_loss_reduction='mean',
                                       pad_token_id= processor.tokenizer.pad_token_id)

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()  # 

In [ ]:
# !apt install git-lfs

In [ ]:
# tokenizer.push_to_hub('wav2vec2tokenizer_sample')
# processor.push_to_hub('processorsample')

In [ ]:
# training hyperparameters in trainingarguments
training_args= TrainingArguments(output_dir='/content/asr_model' ,
                                   group_by_length=True,
                                    per_device_train_batch_size=25,
                                    evaluation_strategy="steps",
                                    num_train_epochs=30,
                                    #fp16=True,
                                    no_cuda = True,
                                    gradient_checkpointing=True,
                                    save_steps=500,
                                    eval_steps=500,
                                    logging_steps=500,
                                    learning_rate=1e-4,
                                    weight_decay=0.005,
                                    warmup_steps=1000,
                                    save_total_limit=2,
                                 push_to_hub= False,
                                 )

In [ ]:
# pass the training arguments to trainer along with the model,dataset, tokenizer, data collator, and compute_metrics
trainer= Trainer(model= model,
                 args= training_args,
                 train_dataset=modified_dataset_['train'],
                 eval_dataset = modified_dataset_['test'],
                 tokenizer = processor.feature_extractor,
                 data_collator= data_collator,
                 compute_metrics=compute_metrics)

In [ ]:
trainer.train()         #

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length_.
***** Running training *****
  Num examples = 2687
  Num Epochs = 30
  Instantaneous batch size per device = 25
  Total train batch size (w. parallel, distributed & accumulation) = 25
  Gradient Accumulation steps = 1
  Total optimization steps = 3240
/usr/local/lib/python3.8/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tensor = as_tensor(value)


In [ ]:
#trainer.push_to_hub()